In [131]:
from pytrends.request import TrendReq
import numpy as np
import pandas as pd

pytrends = TrendReq(hl='en-US', tz=360)



In [3]:
bitcoin_keywords = ['blockchain', 'bitcoin', 'btc', 'cryptocurrency', 'hodl']

In [32]:
a = pytrends.build_payload(bitcoin_keywords, cat=0, timeframe='2017-05-01 2017-08-01', geo='', gprop='')

In [33]:
pytrends.interest_over_time()

,blockchain,bitcoin,btc,cryptocurrency,hodl,isPartial
date,,,,,,
2017-05-01,3,22,8,1,0,False
2017-05-02,4,26,9,1,0,False
2017-05-03,4,26,8,1,0,False
2017-05-04,4,33,10,2,0,False
2017-05-05,4,30,10,2,0,False
2017-05-06,3,25,9,2,0,False
2017-05-07,3,25,8,2,0,False
2017-05-08,4,28,9,2,0,False
2017-05-09,5,37,11,2,0,False


# On tente de faire quelque chose

In [183]:
def google_trends_tables(keyword):
    
    # Google Trends ne permet pas d'avoir une granularité par jour sur une période d'un an.
    #
    # Pour régler le problème, je bricole un peu :
    #  - Je fais une première période qui va du 1er mai 2017 au 1er janvier 2018 (bizarrement, ça marche),
    #  - Et une deuxième période qui va du 1er octobre 2017 au 1er mai 2018.
    #
    # J'utiliserai la période de chevauchement (trois mois) pour déterminer un scale moyen, que j'applique à la
    # colonne de nouvelles valeurs.
    #
    # Les valeurs sont pas vraiment les mêmes (Google s'amuse probablement avec du sampling), mais c'est pas
    # si mal.
    
    keyword = [keyword] # La méthode build_payload demande d'avoir un objet itérable (donc liste)
    
    pytrends_before, pytrends_after = TrendReq(hl = 'en-US', tz = 0), TrendReq(hl = 'en-US', tz = 0)
    
    # Déterminer les modèles pytrends
    pytrends_before.build_payload(keyword, cat = 0, timeframe = '2017-05-01 2018-01-01', geo = '', gprop = '')
    pytrends_after.build_payload(keyword, cat=0, timeframe='2017-10-01 2018-05-01', geo='', gprop='')
    
    # On obtient les tables correspondantes
    table_before = pytrends_before.interest_over_time()
    table_after = pytrends_after.interest_over_time()
    
    return (table_before, table_after)

def scale_generator(table_before, table_after):
    
    # Fonction qui permet de scale
    
    common_table = table_before.merge(table_after, "inner", right_index = True, left_index = True)
    common_table.columns = ['before_value', 'filler1', 'after_value', 'filler2']
    common_table['scale'] = common_table.before_value / common_table.after_value
        
    return common_table.scale.mean()
    
    
def google_trends_common(keyword):
    
    # Génère une grosse table qui fusionne un peu les deux tables before et after
    
    tables = google_trends_tables(keyword)
    table_before, table_after = tables[0], tables[1]
    
    # On renomme les colonnes juste avec le mot clé
    table_before.columns = [keyword, "isPartial"]
    table_after.columns = [keyword, "isPartial"]
    
    # On applique le scale sur la deuxième table
    table_after[keyword] = table_after[keyword].apply(lambda x: int(scale_generator(table_before, table_after) * x), 1)
    
    return table_before.merge(table_after, "outer", right_index = True, left_index = True)


def trends_merge(keyword, min_length = 93):
    
    # Simplifie pour obtenir un truc un peu plus petit
    
    common_table = google_trends_common(keyword)
    
    x_values, y_values = common_table[keyword + '_x'], common_table[keyword + '_y']
    
    # Annuler les valeurs nulles
    x_values = x_values[~np.isnan(x_values)]
    y_values = y_values[~np.isnan(y_values)][min_length:]
    
    merged_values = list(x_values) + list(y_values)    
    merged_values = [value * 100 // max(merged_values) for value in merged_values]
    
    return merged_values
    
    
def google_trends_unique(keyword, min_length = 93):
    
    # Fonction finale à retenir pour l'utilisateur
    
    common_table = google_trends_common(keyword)

    common_table[keyword] = trends_merge(keyword, min_length)
    
    common_table.drop([keyword + '_x', keyword + '_y', "isPartial" + "_x", "isPartial" + "_y"], 1, inplace = True)
    
    return common_table
    

def google_trends(keywords, min_length = 93):
    
    trends = google_trends_unique(keywords[0], min_length)
    
    for keyword in keywords[1:]:
        
        trends = trends.merge(google_trends_unique(keyword), 'left', left_index = True, right_index = True)
        
    return trends

## Ça donne quoi en pratique ?

In [186]:
google_trends(['bitcoin', 'btc', 'cryptocurrency', 'hodl', 'blockchain']).to_csv('trends.csv')

In [182]:
trends.merge(google_trends('btc'), 'left', left_index = True, right_index = True)

,bitcoin,btc
date,,
2017-05-01,4.0,9.0
2017-05-02,4.0,10.0
2017-05-03,4.0,10.0
2017-05-04,5.0,12.0
2017-05-05,5.0,10.0
2017-05-06,4.0,10.0
2017-05-07,4.0,10.0
2017-05-08,5.0,11.0
2017-05-09,6.0,13.0


In [106]:
scale_generator(google_trends_tables('bitcoin')[0], google_trends_tables('bitcoin')[1])

3.5037191802400924

In [93]:
a.apply(lambda x: int(1.18 * x), 1)

date
2017-05-01      2
2017-05-02      1
2017-05-03      1
2017-05-04      1
2017-05-05      2
2017-05-06      0
2017-05-07      1
2017-05-08      2
2017-05-09      1
2017-05-10      1
2017-05-11      0
2017-05-12      1
2017-05-13      0
2017-05-14      1
2017-05-15      1
2017-05-16      1
2017-05-17      1
2017-05-18      1
2017-05-19      2
2017-05-20      2
2017-05-21      1
2017-05-22      1
2017-05-23      2
2017-05-24      3
2017-05-25      5
2017-05-26      3
2017-05-27      3
2017-05-28      1
2017-05-29      1
2017-05-30      4
             ... 
2017-12-03     11
2017-12-04      9
2017-12-05      9
2017-12-06     17
2017-12-07     42
2017-12-08     35
2017-12-09     31
2017-12-10     29
2017-12-11     25
2017-12-12     46
2017-12-13     40
2017-12-14     29
2017-12-15     30
2017-12-16     29
2017-12-17     29
2017-12-18     29
2017-12-19     53
2017-12-20     43
2017-12-21     42
2017-12-22    118
2017-12-23     66
2017-12-24     43
2017-12-25     35
2017-12-26     24
2017-

In [92]:
a.drop(["isPartial"], 1, inplace = True)
a

,hodl
date,
2017-05-01,2
2017-05-02,1
2017-05-03,1
2017-05-04,1
2017-05-05,2
2017-05-06,0
2017-05-07,1
2017-05-08,2
2017-05-09,1
